In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../vector_store/sqlite/Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

: 

## Using LLAMA 3.1 70B Model

In [13]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama-3.1-70b-instruct")



ValidationError: 1 validation error for _NVIDIAClient
base_url
  Input should be a valid string [type=string_type, input_value=FieldInfo(annotation=None... listing an invocation'), input_type=FieldInfo]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type

## Getting queries from SQL DB

In [4]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)

NameError: name 'llm' is not defined

In [5]:
chain.get_prompts()[0].pretty_print()


NameError: name 'chain' is not defined

In [6]:
response = chain.invoke({"question": "How many employees are there"})
print(response)

NameError: name 'chain' is not defined

## Using Agents

In [7]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

NameError: name 'llm' is not defined